# Simple Stateful Chatbot with LangGraph and ChatAmazonNova

This notebook demonstrates building a basic stateful chatbot using LangGraph and Amazon Nova.

The chatbot maintains conversation history and uses LangGraph's state management to provide contextual responses.

## Key Concepts
- **MessagesState**: Built-in state for message tracking
- **State Persistence**: Conversation history maintained across turns
- **History Management**: Automatic trimming to manage context window

## Setup

First, ensure you have the required packages installed and environment variables set.

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

env: NOVA_API_KEY=8c617a8c-823d-42b8-bbc4-5cee38b8f873
env: NOVA_BASE_URL=https://api.nova.amazon.com/v1/


## Import Dependencies

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState

from langchain_amazon_nova import ChatAmazonNova

## Define the Chatbot Node

The chatbot node processes messages with conversation history.

In [3]:
def call_model(state: MessagesState, llm) -> dict:
    """Call the model with conversation history."""
    messages = state["messages"]
    
    # Add system message if this is the first turn
    if len(messages) == 1:
        system_msg = SystemMessage(
            content="You are a helpful assistant. Be conversational and remember context from earlier in the conversation."
        )
        messages = [system_msg] + messages
    
    response = llm.invoke(messages)
    
    return {"messages": [response]}

## Create the LangGraph

Build the graph with a single node for the chatbot.

In [4]:
def create_chatbot_graph(llm):
    """Create a simple stateful chatbot graph."""
    workflow = StateGraph(MessagesState)
    
    workflow.add_node("chatbot", lambda state: call_model(state, llm))
    
    workflow.set_entry_point("chatbot")
    workflow.set_finish_point("chatbot")
    
    return workflow.compile()

## Initialize the Model and Chatbot

In [5]:
# Initialize ChatAmazonNova
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.7,
)

# Create chatbot
chatbot = create_chatbot_graph(llm)

# Initialize conversation state
conversation_state = {"messages": []}

print("Chatbot initialized! Ready to chat.")

Chatbot initialized! Ready to chat.


## Chat with the Bot

Now you can have a conversation! The state persists across cells.

In [6]:
# First turn
user_message = "Hi! My name is Alice."

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

You: Hi! My name is Alice.

Assistant: Hello, Alice! It's nice to meet you. How can I assist you today? Are you looking for information on a specific topic, need help with a task, or just want to chat? Let me know how I can help!


In [7]:
# Second turn - bot should remember your name
user_message = "What's my name?"

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

You: What's my name?

Assistant: You've already told me that your name is Alice. If you're looking for information or have any questions or topics you'd like to discuss, feel free to let me know!


In [8]:
# Third turn - continue the conversation
user_message = "Tell me a fun fact about space."

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

You: Tell me a fun fact about space.

Assistant: Certainly! Here's a fun fact about space:

Did you know that a single supernova—the explosion of a massive star—can outshine an entire galaxy for a brief period? Supernovae are among the most energetic events in the universe, releasing more energy in a few seconds than our Sun will emit over its entire 10-billion-year lifetime. They play a crucial role in the distribution of heavy elements throughout the universe, which are then incorporated into new stars, planets, and even life forms.


## View Conversation History

In [9]:
print(f"Conversation has {len(conversation_state['messages'])} messages:\n")

for i, msg in enumerate(conversation_state["messages"]):
    role = "User" if msg.type == "human" else "Assistant"
    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"{i+1}. {role}: {content}\n")

Conversation has 6 messages:

1. User: Hi! My name is Alice.

2. Assistant: Hello, Alice! It's nice to meet you. How can I assist you today? Are you looking for information on ...

3. User: What's my name?

4. Assistant: You've already told me that your name is Alice. If you're looking for information or have any questi...

5. User: Tell me a fun fact about space.

6. Assistant: Certainly! Here's a fun fact about space:

Did you know that a single supernova—the explosion of a m...



## Clear Conversation and Start Fresh

In [10]:
# Reset the conversation
conversation_state = {"messages": []}
print("Conversation cleared!")

Conversation cleared!


## Try It Yourself

Use the cell below to continue chatting!

In [11]:
# Your turn - modify the message below
user_message = "Your message here"

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

You: Your message here

Assistant: Of course! I'm here to help with whatever you need. Whether you have questions, need advice, or just want to chat, feel free to let me know. What's on your mind today?
